In [11]:
def elves():
    with open("day1.txt") as f:
        data=[line.strip() for line in f.readlines()]
        def chunks():
            p = 0
            for d in data:
                if d=='':
                    yield p
                    p = 0
                else:
                    p += int(d)
        return chunks()
day1a = max(elves())

In [12]:
day1b = sum(sorted(elves(), reverse=True)[0:3])

In [13]:
day1a, day1b

(68923, 200044)

In [34]:
def day2b():
    with open("day2.txt") as f:
        guide=[line.strip() for line in f.readlines()]
        guide = list(map(lambda x: tuple(x.split(" ")), guide))
    
    win = {'A':'Y','B':'Z','C':'X'}
    lose = {'A':'Z','B':'X','C':'Y'}
    draw = {'A':'X','B':'Y','C':'Z'}
    scoring = {'X':1,'Y':2,'Z':3}

    score = lambda opp, you: scoring[you] + (6 if win[opp] == you else 3 if draw[opp] == you else 0)
    move = {'X':lose,'Y':draw,'Z':win}
    
    total_basic = sum([score(opp,you) for opp,you in guide])
    total_advanced = sum([score(opp,(move[you])[opp]) for opp,you in guide])

    return total_basic, total_advanced
day2b()

(11906, 11186)

In [64]:
def day3():
    with open("day3.txt") as f:
        rucksacks = [line.strip() for line in f.readlines()]
        make_set = lambda x: set(list(x))
        rucksacks = list(map(lambda x: (make_set(x[:len(x)//2]),make_set(x[len(x)//2:])), rucksacks))
    priority = lambda p: ord(p) - ord('a') + 1 if p.islower() else ord(p) - ord('A') + 27

    s = 0
    for l,r in rucksacks:
        common = l.intersection(r)
        s += priority(min(common))
    print(s)
    
    s = 0
    groups = [[rucksacks[i],rucksacks[i+1],rucksacks[i+2]] for i in range(0,len(rucksacks),3)]
    for group in groups:
        m = None
        for l,r in group:
            m = l.union(r) if m is None else m.intersection(l.union(r))
        s += priority(min(m))
    print(s)

day3()

7746
2604


In [73]:
def day4():
    with open("day4.txt") as f:
        cfg = [line.strip() for line in f.readlines()]
        inc_range = lambda x,y: range(int(x),int(y)+1)
        mk_range = lambda x: inc_range(*x.split('-'))
        cfg = list(map(lambda c: list(map(set,map(mk_range,c.split(",")))), cfg))
        
    def overlaps(c):
        left,right = tuple(c)
        return len(left.intersection(right)) > 0
    
    def contains(c):
        left,right = tuple(c)
        return len(left.intersection(right)) == min(len(left),len(right))
    
    a = sum(map(contains, cfg))
    b = sum(map(overlaps, cfg))

    print(a,b)
day4()

490 921


In [113]:
import re
def day5():
    with open("day5.txt") as f:
        lines = map(lambda l: l.strip(), f.readlines())
        crates = [[]] * 9
        crates_b = [[]] * 9
        for l in lines:
            for i,s in enumerate(l[1::4]):
                if s.isupper():
                    crates[i] = [s] + crates[i]
                    crates_b[i] = [s] + crates_b[i]
            if l == "":
                break
        pattern = re.compile("move (.*) from (.*) to (.*)")
        for l in lines:
            n, fr, to = map(int, pattern.search(l).groups())
            for _ in range(n):
                c = crates[fr-1].pop()
                crates[to-1].append(c)
            crates_b[to-1] = crates_b[to-1] + crates_b[fr-1][-n:]
            del crates_b[fr-1][-n:]
        print("".join([c[-1] for c in crates]))
        print("".join([c[-1] for c in crates_b]))

day5()

SBPQRSCDF
RGLVRCQSB


In [112]:
list(range(10))[-3:]

[7, 8, 9]